# Programming Assignment # 4 Clustering

## Student Details

When submitting, fill your full name, your student ID and your NetID in this cell. Note that this is a markdown cell! 

Student Full Name: Vijetha Shenoy B

ID:1001822855

Team Mate name :

ID: 

## Rules

1. Work is to be done in a team
2. Any cheating including plagiarism, cooperation will be reported to the corresponding UTA’ s instance.
3. If using any resource (books, internet), please make sure that you cite it.
4. Follow the given structure. Specifically, place all your tasks in THIS NOTEBOOK BUT IN SEPARATE BLOCKS. Then save this notebook as 'yourNetID_pa3.ipynb' and submit it. 
5. Do not alter the dataset name.
6. Please dont ask any details specific to the project like "How to plot XYZ ? What parameters are to be used? " and so on..
7. Report is not required for this assignment. If you want to document a function or a process, just comment or use markup cell.
8. Please dont send images of your visualizations to verify whether they are right or not before submission deadline.



## Assignment Details

The purpose of this assignment is to cluster  using K-means clustering and Hierarchical Agglomerative clustering models and to visualize clusters for predicted and actual cluster labels.


Your dataset is given as 3 files . 




You need to submit this ipython file after renaming it. 

Preprocessing will be needed for the data as most of the data is in string and needs to be quantified.



In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Required Python Packages 

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import *
import seaborn as sns 
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans 
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import PCA
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 

In [3]:
# Importing the Data Set
df = pd.read_csv("clusteringdata.csv")
#print th first 5 rows of the dataset
df.head()

,Age,WorkClass,Fnlwght,Education,EducationNumber,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Removing any space in the names of the columns
df.columns = df.columns.str.replace(' ', '')

In [5]:
# Dropping the duplicate Rows
df = df.drop_duplicates(keep = 'first')

In [6]:
# Checking the null values in the columns
df.isnull().sum(axis = 0)

Age                0
WorkClass          0
Fnlwght            0
Education          0
EducationNumber    0
MaritalStatus      0
Occupation         0
Relationship       0
Race               0
Sex                0
CapitalGain        0
CapitalLoss        0
HoursPerWeek       0
NativeCountry      0
Class              0
dtype: int64

In [7]:
# This Code will Count the occuring of the '?' in all the columns
for i in df.columns:
    t = df[i].value_counts()
    index = list(t.index)
    #print ("The Value Counts of ? in", i)
    for i in index:
        temp = 0
        if i == ' ?':
           # print (t[' ?'])
            temp = 1
            break
    #if temp == 0:
        #print ("0")
# Dropping the rows which have '?' 
df = df[df.WorkClass != ' ?']
df = df[df.Occupation != ' ?']
df = df[df.NativeCountry != ' ?']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4579 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Age              4579 non-null   int64 
 1   WorkClass        4579 non-null   object
 2   Fnlwght          4579 non-null   int64 
 3   Education        4579 non-null   object
 4   EducationNumber  4579 non-null   int64 
 5   MaritalStatus    4579 non-null   object
 6   Occupation       4579 non-null   object
 7   Relationship     4579 non-null   object
 8   Race             4579 non-null   object
 9   Sex              4579 non-null   object
 10  CapitalGain      4579 non-null   int64 
 11  CapitalLoss      4579 non-null   int64 
 12  HoursPerWeek     4579 non-null   int64 
 13  NativeCountry    4579 non-null   object
 14  Class            4579 non-null   object
dtypes: int64(6), object(9)
memory usage: 572.4+ KB


In [8]:
df['Class'] = df['Class'].map({' <=50K':0, ' >50K':1})
df['Class'].value_counts()
# Changing the Categorical Values to Numerical values using the Label Encoder
from sklearn.preprocessing import LabelEncoder

features = list(df.select_dtypes(include=['object']).columns)
le_feat = {}
# using enumerator to keep a count of iterations.
for i, feature in enumerate(features):
    #to converting the labels(feature) into numeric form
    le_feat[feature] = LabelEncoder() 
    #scale the training data and also learn the scaling parameters of that data
    df[feature] = le_feat[feature].fit_transform(df[feature])
df.head()

,Age,WorkClass,Fnlwght,Education,EducationNumber,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Class
0,39,5,77516,9,13,4,0,1,4,1,2174,0,40,36,0
1,50,4,83311,9,13,2,3,0,4,1,0,0,13,36,0
2,38,2,215646,11,9,0,5,1,4,1,0,0,40,36,0
3,53,2,234721,1,7,2,5,0,2,1,0,0,40,36,0
4,28,2,338409,9,13,2,9,5,2,0,0,0,40,4,0


## TASK 1: K-Means Clustering

### Task 1-a: Determine “k” value from the elbow method

In this task, you will be using the elbow method to determine the optimal number of clusters for k-means clustering.

We need some way to determine whether we are using the right number of clusters when using k-means clustering. One method to validate the number of clusters is the elbow method. 

The idea of the elbow method is to run k-means clustering on the dataset for a range of values of k (k will be from 1 to 10 in this task), and for each value of k calculate the sum of squared errors (SSE). Then, plot a line chart of the SSE for each value of k. If the line chart looks like an arm, then the "elbow" on the arm is the value of k that is the best. The idea is that we want a small SSE, but that the SSE tends to decrease toward 0 as we increase k (the SSE is 0 when k is equal to the number of data points in the dataset, because then each data point is a cluster, and there is no error between it and the center of its cluster). So our goal is to choose a small value of k that still has a low SSE, and the elbow usually represents where we start to have diminishing returns by increasing k.

For this task, you need to perform the elbow method for k from 1 to 10 and plot a line chart of the SSE for each value of k, and determine the best k (the number of clusters). Note that you need to use the whole dataset in this task and you need to print your decision for k.

In [ ]:
#Squared Errors
SSE = {} 
for k in range(1, 11):     
    kmeans = KMeans(n_clusters=k, random_state=1)     
    kmeans.fit(df)     
    SSE[k] = kmeans.inertia_ 
# sum of squared distances to closest cluster cente
# Plot SSE for each *k* 
plt.title('The Elbow Method') 
plt.xlabel('k'); 
plt.ylabel('SSE') 
#plt.plot(list(SSE.keys()),list(SSE.values()), color='black', marker=(5,0), markerfacecolor='black', markersize=10)
print("The best value for k in the")
sns.pointplot(x=list(SSE.keys()), y=list(SSE.values())) 
plt.show() 


### Task 1-b: Visualization for K-Means Clustering

In this task, you will be performing k-means clustering for k=2 and visualize the predicted training samples and actual training samples on scatter plots. Use 70% of the dataset for training and 30% of the dataset for testing. Perform kmeans for clustering samples in your training set. 

Use two subplots for visualizing the predicted training samples and actual training samples on two scatter plots.

Since your dataset has multiple features(dimensions), you won't be able to plot your data on a scatter plot. Thus, you’re going to visualize your data with the help of one of the Dimensionality Reduction techniques, namely Principal Component Analysis (PCA). The idea in PCA is to find a linear combination of the two variables that contains most of the information. This new variable or “principal component” can replace the two original variables. You can easily apply PCA to your data with the help of scikit-learn.



In [ ]:
###################begin code for Task 1-b-1: Split the dataset 70% for training and 30% for testing
C1 = df.drop(columns = ['Class'])
C2 = df['Class']
X_train, X_test, Y_train, Y_test = train_test_split(C1, C2, test_size = 0.3,random_state = 100)
###################end code for Task 1-b-1

In [ ]:

###################begin code for Task 1-b-2: Visualize the predicted training labels vs actual training labels

# Import PCA

# Create the KMeans model
kmeans = KMeans(n_clusters= 2)
# fit our model using our training data.
kmeans.fit(X_train, Y_train)

# Compute cluster centers and predict cluster index for each sample 
clusters = kmeans.fit_predict(X_train)

# Model and fit the data to the PCA model
pca = PCA()
X_train_pca = pca.fit_transform(X_train)

# Visualize the predicted training labels vs actual training labels. 
# scatter(x, y, your_data)
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
# Adjust layout
fig.suptitle('Predicted Versus Training Labels', fontsize=14, fontweight='bold')
fig.subplots_adjust(top=0.9)

# Add scatterplots to the subplots 
x = X_train_pca[:, 0]
y = X_train_pca[:, 1]
s=ax[0].scatter(x, y, c=clusters)
ax[0].set_title('Predicted Training Labels')
ax[1].scatter(x, y, c=Y_train)
ax[1].set_title('Actual Training Labels')

for a in ax.flat:
    a.set(xlabel='First PCA component', ylabel='Second PCA component')
    legend1 = a.legend(*s.legend_elements(),
                    loc="upper right", title="Classes")
    a.add_artist(legend1)

plt.show()

###################end code for Task 1-b-2

Now, you need to visualize the predicted testing labels versus actual testing labels. Use the trained model in previous step.

In [ ]:
###################begin code for Task 1-b-3: Visualize the predicted testing labels vs actual testing labels
# Create the KMeans model
kmeans_val = KMeans(n_clusters= 2)

# predict cluster index for each sample 
cluster_index = kmeans_val.fit_predict(X_test)

# Model and fit the data to the PCA model
pca = PCA()
#X_train_pca = pca.fit_transform(X_test)
X_train_pca = PCA(n_components=2).fit_transform(X_test)

# Visualize the predicted testing labels vs actual testing labels. 
img, ax = plt.subplots(1, 2, figsize=(15, 6))
# Adjust layout
fig.suptitle('Predicted Versus Testing Labels', fontsize=14, fontweight='bold')
fig.subplots_adjust(top=0.9)

# Add scatterplots to the subplots 
x = X_train_pca[:, 0]
y = X_train_pca[:, 1]
s = ax[0].scatter(x, y, c=cluster_index)
ax[0].set_title('Predicted Testing Labels')
ax[1].scatter(x, y, c=Y_test)
ax[1].set_title('Actual Testing Labels')

for a in ax.flat:
    a.set(xlabel='First PCA component', ylabel='Second PCA component')
    legend1 = a.legend(*s.legend_elements(),loc="upper right", title="Classes")
    a.add_artist(legend1)
plt.show()

###################end code for Task 1-b-3

In this step, you need to provide the evaluation of your clustering model. Print out a confusion matrix.

In [ ]:
###################begin code for Task 1-b-4: Print out a confusion matrix

Y_prediction = kmeans.predict(X_test)
print("Confusion Matrix: \n",confusion_matrix(Y_test,Y_prediction),'\n')
print("Classification Report: \n",classification_report(Y_test,Y_prediction))

###################end code for Task 1-b-4

# TASK 2: Hierarchical Agglomerative  Clustering

## Task 2-a: Find the best Hierarchical Agglomerative Clustering Model

In this task, you will be performing Hierarchical Agglomerative clustering with different linkage methods (complete and average) and different similarity measures (cosine, euclidean, and manhattan) in order to find the best pair of linkage method and similarity measure. Use F1 score for evaluation and take n_clusters = 2.


In [ ]:
###################begin code for Task 2-a: Print out a confusion matrix

## Calculate pairwise distance matrix for X_train
pdm_train = pairwise_distances(X_train)

## Model and fit the training data to the AgglomerativeClustering model
## complete linkage + cosine
complete_cosine = AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage="complete")
prediction1 = complete_cosine.fit_predict(X_train)

## Model and fit the training data to the AgglomerativeClustering model
## complete linkage + euclidean
complete_euclidean = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage="complete")
prediction2 = complete_euclidean.fit_predict(X_train)

## Model and fit the training data to the AgglomerativeClustering model
## complete linkage + manhattan
complete_manhattan = AgglomerativeClustering(n_clusters=2, affinity='manhattan', linkage="complete")
prediction3 = complete_manhattan.fit_predict(X_train)

## Model and fit the training data to the AgglomerativeClustering model
## average linkage + cosine
average_cosine = AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage="average")
prediction4 = average_cosine.fit_predict(X_train)

## Model and fit the training data to the AgglomerativeClustering model
## average linkage + euclidean
average_euclidean = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage="average")
prediction5 = average_euclidean.fit_predict(X_train)

## Model and fit the training data to the AgglomerativeClustering model
## average linkage + manhattan
average_manhattan = AgglomerativeClustering(n_clusters=2, affinity='manhattan', linkage="average")
prediction6 = average_manhattan.fit_predict(X_train)

print("F1-score for complete linkage + cosine",f1_score(Y_train, prediction1))
print("F1-score for complete linkage + euclidean", f1_score(Y_train, prediction2))
print("F1-score for complete linkage + manhattan",f1_score(Y_train, prediction3))
print("F1-score for average linkage + cosine", f1_score(Y_train, prediction4))
print("F1-score for average linkage + euclidean", f1_score(Y_train, prediction5))
print("F1-score for average linkage + manhattan", f1_score(Y_train, prediction6))

###################end code for Task 2-a

## Task 2-b:  Visualization for Hierarchical Agglomerative Clustering

Find the best performed model from the previous step and use that model for visualizing the predicted training samples and actual training samples on scatter plots. Use PCA model for visualizing your data (use X_train_pca from Task 1-b-2). 

In [ ]:
###################begin code for Task 2-b: Visualize the predicted training labels vs actual training labels

# Model and fit the data to the PCA model
pca = PCA()
X_train_pca = pca.fit_transform(X_train)

# Visualize the predicted training labels vs actual training labels. 

fig, ax = plt.subplots(1, 2, figsize=(15, 6))
img.suptitle('Predicted Versus Training Labels', fontsize=14, fontweight='bold')
img.subplots_adjust(top=0.9)

# Add scatterplots to the subplots 
x = X_train_pca[:, 0]
y = X_train_pca[:, 1]
s=ax[0].scatter(x, y, c=prediction2)
ax[0].set_title('Predicted Training Labels')
ax[1].scatter(x, y, c=Y_train)
ax[1].set_title('Actual Training Labels')
for a in ax.flat:
    a.set(xlabel='First PCA component', ylabel='Second PCA component')
    legend1 = a.legend(*s.legend_elements(),
                    loc="upper right", title="Classes")
    a.add_artist(legend1)
plt.show()

###################end code for Task 2-b

# TASK 3:  Compare K-Means Clustering and Hierarchical Agglomerative Clustering

## Task 3-a: Visualize Clusters 

In this task, use whole dataset for training k-means cluster and hierarchical agglomerative clustering. Use the best model for agglomerative clustering. Visualize the predicted labels from k-means clustering and agglomerative clustering versus actual labels. Basically, you need to plot three scatter plots as subplots. 

In [ ]:
## Model and fit the training data to the AgglomerativeClustering model
## complete linkage + cosine
X = df[df.columns[:-1]]
Y = df[df.columns[-1]]
complete_cosine = AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage="complete")
pred1 = complete_cosine.fit_predict(X)


## Model and fit the training data to the AgglomerativeClustering model
## complete linkage + euclidean
complete_euclidean = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage="complete")
pred2 = complete_euclidean.fit_predict(X)

## Model and fit the training data to the AgglomerativeClustering model
## complete linkage + manhattan
complete_manhattan = AgglomerativeClustering(n_clusters=2, affinity='manhattan', linkage="complete")
pred3 = complete_manhattan.fit_predict(X)

## Model and fit the training data to the AgglomerativeClustering model
## average linkage + cosine
average_cosine = AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage="average")
pred4 = average_cosine.fit_predict(X)

## Model and fit the training data to the AgglomerativeClustering model
## average linkage + euclidean
average_euclidean = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage="average")
pred5 = average_euclidean.fit_predict(X)

## Model and fit the training data to the AgglomerativeClustering model
## average linkage + manhattan
average_manhattan = AgglomerativeClustering(n_clusters=2, affinity='manhattan', linkage="average")
pred6 = average_manhattan.fit_predict(X)

print("F1-score for complete linkage + cosine",f1_score(Y, pred1))
print("F1-score for complete linkage + euclidean", f1_score(Y, pred2))
print("F1-score for complete linkage + manhattan",f1_score(Y, pred3))
print("F1-score for average linkage + cosine", f1_score(Y, pred4))
print("F1-score for average linkage + euclidean", f1_score(Y, pred5))
print("F1-score for average linkage + manhattan", f1_score(Y, pred6))

In [ ]:
###################begin code for Task 3-a: Visualize the predicted training labels vs actual training labels

### Kmeans Clustering
# Model and fit the data to the Kmeans (use fit_predict : Performs clustering on X and returns cluster labels.)
X = df.drop(columns = ['Class'])
Y = df['Class']
kmeans = KMeans(n_clusters=2,random_state=100)
kmeans_prediction = kmeans.fit_predict(X)

### Agglomerative Clustering
# Calculate pairwise distance matrix for X
pdm_train = pairwise_distances(X)
# print(pdm_train)

# Model and fit the data to the Agglomerative (use fit_predict : Performs clustering on X and returns cluster labels.)
Agg_clustering = AgglomerativeClustering(n_clusters=2)
Agg_prediction = Agg_clustering.fit_predict(X)

#  Model and fit the data to the PCA model
pca = PCA()
X_pca = pca.fit_transform(X)

# Visualize the predicted Kmeans labels versus  the predicted Agglomerative labels versus Actual labels. 
# scatter(x, y, your_data)
fig1, ax = plt.subplots(1, 2,figsize=(15, 6))
fig, axs = plt.subplots(figsize=(15, 6))


# Adjust layout
fig1.suptitle('predicted Kmeans labels versus  the predicted Agglomerative labels versus Actual labels', fontsize=14, fontweight='bold')
fig1.subplots_adjust(top=0.9)

# Add scatterplots to the subplots 
x = X_pca[:, 0]
y = X_pca[:, 1]
s=ax[0].scatter(x, y, c=kmeans_prediction)
ax[0].set_title('Predicted Kmeans labels')
ax[1].scatter(x, y, c=pred4)
ax[1].set_title('Predicted Agglomerative labels')
s=axs.scatter(x, y, c=Y)
axs.set_title('Actual Training Labels')

for a in ax.flat:
    a.set(xlabel='First PCA component', ylabel='Second PCA component')
    legend1 = a.legend(*s.legend_elements(),
                    loc="upper right", title="Classes")
    a.add_artist(legend1)

plt.show()

###################end code for Task 3-a

## Task 3-b: Compare K-Means Clustering &  Hierarchical Agglomerative Clustering

Print out confusion matrices for kmeans and agglomerative clustering. Also, compare precision, recall, and F1-score for both model. Type your reasoning.

In [ ]:
###################begin code for Task 3-b
Y_kmeans_pred = kmeans.predict(X)
print("Kmeans:\n Accuracy is:", accuracy_score(Y,Y_kmeans_pred)*100,"\n")
print("Confusion Matrix: \n",confusion_matrix(Y,Y_kmeans_pred),'\n')
print("Classification Report: \n",classification_report(Y,Y_kmeans_pred))

Y_Agglemorative_pred = Agg_clustering.fit_predict(X)
print("Agglomerative clustering: \nAccuracy is:", accuracy_score(Y,Y_Agglemorative_pred)*100,"\n")
print("Confusion Matrix: \n",confusion_matrix(Y,Y_Agglemorative_pred),'\n')
print("Classification Report: \n",classification_report(Y,Y_Agglemorative_pred))
###################end code for Task 3-b

## Grading 

<b>[05 points] Follow the Rules</b> 


<b>[45 points] Task 1:</b>  

    [15 points] Task 1-a: Determine “k” value from the elbow method

    [30 points] Task 1-b: Visualization for K-Means Clustering

        [05 points] Task 1-b-1: Split the dataset 
    
        [10 points] Task 1-b-2: Visualize the predicted training vs actual training labels 
    
        [10 points] Task 1-b-3: Visualize the predicted testing vs actual testing labels
    
        [05 points] Task 1-b-4: Print out a confusion matrix
    


<b>[50 points] Task 2:</b>  

    [35 points] Task 2-a: Find the best Hierarchical Agglomerative Clustering Model

    [15 points] Task 2-b: Visualization for Hierarchical Agglomerative Clustering



<b>[20 points] Task 3 (BONUS):</b> 

    Task 3-a: 10 points

    Task 3-b: 10 points